In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Prepare data

In [2]:
flights = pd.read_excel("/content/drive/MyDrive/Kho dữ liệu và OLAP/US_flights_2023.xlsx")
# flights = flights.head(50000)

In [3]:
dep_airports = pd.read_excel("/content/drive/MyDrive/Kho dữ liệu và OLAP/airports_geolocation.xlsx")
arr_airports = pd.read_excel("/content/drive/MyDrive/Kho dữ liệu và OLAP/airports_geolocation.xlsx")

In [4]:
weather = pd.read_excel("/content/drive/MyDrive/Kho dữ liệu và OLAP/weather_meteo_by_airport.xlsx")

In [5]:
flights.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Flight_Duration', 'Distance_type', 'Delay_Carrier',
       'Delay_Weather', 'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft',
       'Manufacturer', 'Model', 'Aicraft_age'],
      dtype='object')

In [6]:
dep_airports.columns

Index(['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [7]:
sorted_df = flights.sort_values(by=['Dep_Airport', 'Arr_Airport'])
sorted_df[['Dep_Airport', 'Arr_Airport', 'Distance_type']]

,Dep_Airport,Arr_Airport,Distance_type
18101,ABE,ATL,Short Haul >1500Mi
24123,ABE,ATL,Short Haul >1500Mi
28258,ABE,ATL,Short Haul >1500Mi
31697,ABE,ATL,Short Haul >1500Mi
38041,ABE,ATL,Short Haul >1500Mi
...,...,...,...
74044,YUM,PHX,Short Haul >1500Mi
80404,YUM,PHX,Short Haul >1500Mi
84227,YUM,PHX,Short Haul >1500Mi
88847,YUM,PHX,Short Haul >1500Mi


## Merge aiport info

In [8]:
merged_dep_airport = pd.merge(flights, dep_airports, left_on=['Dep_Airport'], right_on=['IATA_CODE'], how='inner')
merged_dep_airport.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Flight_Duration', 'Distance_type', 'Delay_Carrier',
       'Delay_Weather', 'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft',
       'Manufacturer', 'Model', 'Aicraft_age', 'IATA_CODE', 'AIRPORT', 'CITY',
       'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [9]:
merged_dep_airport = merged_dep_airport.drop(['Flight_Duration', 'Delay_Carrier','Delay_Weather', 'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft',\
                         'IATA_CODE', 'AIRPORT'], axis=1)
col_mapping = {
    'CITY': 'dep_city',
    'STATE': 'dep_state',
    'COUNTRY': 'dep_country',
    'LATITUDE': 'dep_latitude',
    'LONGITUDE': 'dep_longitude'
}

merged_dep_airport = merged_dep_airport.rename(columns=col_mapping)

In [10]:
merged_dep_airport.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude'],
      dtype='object')

In [11]:
merged_arr_airport = pd.merge(merged_dep_airport, arr_airports, left_on=['Arr_Airport'], right_on=['IATA_CODE'], how='inner')
merged_arr_airport.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude', 'IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY',
       'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [12]:
merged_arr_airport = merged_arr_airport.drop(['IATA_CODE', 'AIRPORT'], axis=1)
col_mapping = {
    'CITY': 'arr_city',
    'STATE': 'arr_state',
    'COUNTRY': 'arr_country',
    'LATITUDE': 'arr_latitude',
    'LONGITUDE': 'arr_longitude'
}
merged_arr_airport = merged_arr_airport.rename(columns=col_mapping)
merged_arr_airport.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude', 'arr_city', 'arr_state', 'arr_country', 'arr_latitude',
       'arr_longitude'],
      dtype='object')

## Merge weather info

In [13]:
weather.columns

Index(['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres',
       'airport_id'],
      dtype='object')

In [14]:
merged_dep_weather = pd.merge(merged_arr_airport, weather, left_on=['FlightDate', 'Dep_Airport'], right_on=['time', 'airport_id'], how='inner')

In [15]:
merged_dep_weather = merged_dep_weather.drop(['time', 'airport_id'], axis=1)
col_mapping = {
    'tavg': 'dep_tavg',
    'tmin': 'dep_tmin',
    'tmax': 'dep_tmax',
    'prcp': 'dep_prcp',
    'snow': 'dep_snow',
    'wdir': 'dep_wdir',
    'wspd': 'dep_wspd',
    'pres': 'dep_pres',
}
merged_dep_weather = merged_dep_weather.rename(columns=col_mapping)
merged_dep_weather.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude', 'arr_city', 'arr_state', 'arr_country', 'arr_latitude',
       'arr_longitude', 'dep_tavg', 'dep_tmin', 'dep_tmax', 'dep_prcp',
       'dep_snow', 'dep_wdir', 'dep_wspd', 'dep_pres'],
      dtype='object')

In [16]:
merged_arr_weather = pd.merge(merged_dep_weather, weather, left_on=['FlightDate', 'Arr_Airport'], right_on=['time', 'airport_id'], how='inner')

In [17]:
merged_arr_weather = merged_arr_weather.drop(['time', 'airport_id'], axis=1)
col_mapping = {
    'tavg': 'arr_tavg',
    'tmin': 'arr_tmin',
    'tmax': 'arr_tmax',
    'prcp': 'arr_prcp',
    'snow': 'arr_snow',
    'wdir': 'arr_wdir',
    'wspd': 'arr_wspd',
    'pres': 'arr_pres',
}
merged_arr_weather = merged_arr_weather.rename(columns=col_mapping)
merged_arr_weather.columns

Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
       'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
       'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude', 'arr_city', 'arr_state', 'arr_country', 'arr_latitude',
       'arr_longitude', 'dep_tavg', 'dep_tmin', 'dep_tmax', 'dep_prcp',
       'dep_snow', 'dep_wdir', 'dep_wspd', 'dep_pres', 'arr_tavg', 'arr_tmin',
       'arr_tmax', 'arr_prcp', 'arr_snow', 'arr_wdir', 'arr_wspd', 'arr_pres'],
      dtype='object')

## Extract day, month, year

In [18]:
# Splitting the date string and converting to integers
merged_arr_weather['Year'] = merged_arr_weather['FlightDate'].str.split('-').str[0].astype(int)
merged_arr_weather['Month'] = merged_arr_weather['FlightDate'].str.split('-').str[1].astype(int)
merged_arr_weather['Day'] = merged_arr_weather['FlightDate'].str.split('-').str[2].astype(int)

merged_arr_weather.drop(['FlightDate'], axis=1, inplace=True)

In [19]:
merged_arr_weather.drop(['Dep_CityName', 'Arr_CityName'], axis=1, inplace=True)

In [20]:
merged_arr_weather.columns

Index(['Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport', 'DepTime_label',
       'Dep_Delay', 'Dep_Delay_Tag', 'Dep_Delay_Type', 'Arr_Airport',
       'Arr_Delay', 'Arr_Delay_Type', 'Distance_type', 'Manufacturer', 'Model',
       'Aicraft_age', 'dep_city', 'dep_state', 'dep_country', 'dep_latitude',
       'dep_longitude', 'arr_city', 'arr_state', 'arr_country', 'arr_latitude',
       'arr_longitude', 'dep_tavg', 'dep_tmin', 'dep_tmax', 'dep_prcp',
       'dep_snow', 'dep_wdir', 'dep_wspd', 'dep_pres', 'arr_tavg', 'arr_tmin',
       'arr_tmax', 'arr_prcp', 'arr_snow', 'arr_wdir', 'arr_wspd', 'arr_pres',
       'Year', 'Month', 'Day'],
      dtype='object')

# Classification

In [78]:
data = merged_arr_weather.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 44 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Day_Of_Week     100000 non-null  int64  
 1   Airline         100000 non-null  object 
 2   Tail_Number     100000 non-null  object 
 3   Dep_Airport     100000 non-null  object 
 4   DepTime_label   100000 non-null  object 
 5   Dep_Delay       100000 non-null  int64  
 6   Dep_Delay_Tag   100000 non-null  int64  
 7   Dep_Delay_Type  100000 non-null  object 
 8   Arr_Airport     100000 non-null  object 
 9   Arr_Delay       100000 non-null  int64  
 10  Arr_Delay_Type  100000 non-null  object 
 11  Distance_type   100000 non-null  object 
 12  Manufacturer    100000 non-null  object 
 13  Model           100000 non-null  object 
 14  Aicraft_age     100000 non-null  int64  
 15  dep_city        100000 non-null  object 
 16  dep_state       100000 non-null  object 
 17  dep_country

In [79]:
column_names = data.columns
categorical_variables = {}

# Iterate through columns
for col in column_names:
  categorical_variables[col] = data[col].unique()


# Print the formatted output
print("categorical_variables = {")
for col, values in categorical_variables.items():
    print(f'    "{col}": {values},')
print("}")

categorical_variables = {
    "Day_Of_Week": [5 2 3 6 4 1 7],
    "Airline": ['Skywest Airlines Inc.' 'American Eagle Airlines Inc.'
 'American Airlines Inc.' 'Republic Airways' 'Endeavor Air'
 'Southwest Airlines Co.' 'United Air Lines Inc.' 'JetBlue Airways'
 'Delta Air Lines Inc' 'Spirit Air Lines' 'Frontier Airlines Inc.'
 'Alaska Airlines Inc.' 'Hawaiian Airlines Inc.' 'PSA Airlines'
 'Allegiant Air'],
    "Tail_Number": ['N717EV' 'N263NN' 'N938NN' ... 'N37298' 'N36280' 'N77295'],
    "Dep_Airport": ['DFW' 'PIT' 'DCA' 'IND' 'CMH' 'CLE' 'DAL' 'BGM' 'LAS' 'PHX' 'LGA' 'TPA'
 'EWR' 'IAH' 'DTW' 'SEA' 'FAT' 'BOI' 'XWA' 'ATL' 'AUS' 'HNL' 'ORD' 'RAP'
 'BIS' 'MEM' 'DEN' 'BOS' 'SYR' 'SAN' 'BWI' 'MCO' 'CLT' 'FLL' 'PBI' 'SDF'
 'SAV' 'PWM' 'RIC' 'MIA' 'SJU' 'JFK' 'MCI' 'TUL' 'MDW' 'LGB' 'SGF' 'MRY'
 'ANC' 'PHL' 'TUS' 'OAK' 'SMF' 'MSP' 'SFO' 'LAX' 'CVG' 'IAD' 'ELP' 'CID'
 'BUF' 'SNA' 'MKE' 'TVC' 'JAX' 'KOA' 'PDX' 'ROC' 'RDU' 'OMA' 'ABQ' 'FCA'
 'OGG' 'EUG' 'BNA' 'ASE' 'HOU' 'SLC' 'AVP' 'TYS' 'PN

In [104]:
df1 = data.copy()

In [105]:
# map values in the DepTime_label column by this strategy
# 'Evening':3, 'Morning':1, 'Afternoon':2, 'Night':4
df1['DepTime_label'] = df1['DepTime_label'].map({
    'Morning': 1,
    'Afternoon': 2,
    'Evening': 3,
    'Night': 4
})

In [106]:
# Separate the features (X) and the target (y)
X = df1[['DepTime_label', 'Day_Of_Week', 'dep_pres', 'arr_wdir', 'arr_pres']]
y = df1['Dep_Delay_Tag']

# Split the data into a test subset (smaller portion) and a training subset (larger portion)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [107]:
numerical_columns = X.select_dtypes(exclude = 'object').columns
categorical_columns = X_train.select_dtypes(include = 'object').columns
categorical_column_indices = [X_train.columns.get_loc(col) for col in categorical_columns]

In [109]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42)

# Apply SMOTE to the training data
X_train, y_train = adasyn.fit_resample(X_train[numerical_columns], y_train)

In [125]:
X_train

,DepTime_label,Day_Of_Week,dep_pres,arr_wdir,arr_pres
0,1,2,1012.200000,217.000000,1006.200000
1,3,1,1014.300000,85.000000,1020.900000
2,1,7,1028.800000,159.000000,1008.700000
3,2,4,1010.000000,12.000000,1013.300000
4,3,3,1018.000000,46.000000,1019.000000
...,...,...,...,...,...
100122,3,6,1015.187018,294.145029,1017.443509
100123,2,5,1015.147793,189.761036,1011.552207
100124,1,3,1009.255554,328.000000,1010.400000
100125,1,3,1010.500000,212.000000,1008.800000


In [123]:
# Additional classifiers for models_without_categorical
model = KNeighborsClassifier()

In [124]:
# iterate k from 3 to 10 in order to select the best model
for k in range(3, 11):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"k = {k}, accuracy = {accuracy}")
    print(classification_report(y_test, y_pred))
    print()

k = 3, accuracy = 0.52475
              precision    recall  f1-score   support

           0       0.64      0.52      0.58     12423
           1       0.40      0.53      0.46      7577

    accuracy                           0.52     20000
   macro avg       0.52      0.53      0.52     20000
weighted avg       0.55      0.52      0.53     20000


k = 4, accuracy = 0.56035
              precision    recall  f1-score   support

           0       0.64      0.68      0.66     12423
           1       0.41      0.37      0.39      7577

    accuracy                           0.56     20000
   macro avg       0.52      0.52      0.52     20000
weighted avg       0.55      0.56      0.56     20000


k = 5, accuracy = 0.5242
              precision    recall  f1-score   support

           0       0.65      0.51      0.57     12423
           1       0.40      0.54      0.46      7577

    accuracy                           0.52     20000
   macro avg       0.53      0.53      0.52     2

In [120]:
# reset X_test index and merge y_pred to X_test
X_test_df = X_test.reset_index(drop=True)
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
X_test_df = pd.concat([X_test_df, y_pred_df], axis=1)

# save to excel
X_test_df.to_excel('X_test_df.xlsx', index=False)